# Playing with generator

In [1]:
#%%timeit
import glob
import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
import numpy as np
import sys
import copy
from pathlib import Path
#from src.envs.generators import RandomRecurrence
#from src.envs.encoders import Equation, IntegerSeries, RealSeries
from sympy import *
import pickle
from collections import defaultdict, OrderedDict
import math

%reload_ext autoreload

#%cd ~/recur/notebooks
sys.path.append('/private/home/pakamienny/Research_2/recur')
import src
from src.envs.generators import RandomRecurrence
from src.envs.encoders import Equation, RealSeries, IntegerSeries
from src.envs.recurrence import RecurrenceEnvironment


In [2]:
class params:
    float_precision = 3
    max_len = 30
    max_degree = 6
    max_number = 1e100
    real_series = True
    output_numeric=False
    max_exponent = 100
    init_scale = 10
    prob_const=1/3
    prob_n = 1/3
    max_int = 10
    prob_rand = 0.05
    dimension=1
    int_base=10000
    n_predictions=5
    float_tolerance=0.1
    more_tolerance="0.1"
    max_ops=10
    max_token_len=200
    eval_noise=False
    eval_input_length_modulo = -1
    operators_to_remove=""
    required_operators=""

generator = RandomRecurrence(params)
env = RecurrenceEnvironment(params)
env.rng = np.random.RandomState(0)
import time 

SPECIAL_WORDS = ["EOS", "PAD", "(", ")", "SPECIAL"]

if params.real_series:
    input_encoder = RealSeries(params)
else:
    input_encoder = IntegerSeries(params)
input_words = SPECIAL_WORDS+sorted(list(set(input_encoder.symbols)))

if params.output_numeric:
    if params.real_series:
        output_encoder = RealSeries(params)
    else:
        output_encoder = IntegerSeries(params)
else:
    output_encoder = Equation(params)

deb = time.time()
k_trials=10000
successful_trials=0
size_input=[]
size_output=[]
infos_stats = {}
#n_ops_stats = {i:0 for i in range(1,(params.max_ops+1)*params.dimension)}

for k in range(k_trials):
    
    x, y, tree, infos = env.gen_expr(False, params.eval_input_length_modulo)
    if tree is not None:
        for info in infos:
            if info not in infos_stats:
                infos_stats[info]=np.zeros((1000,))
            for val in infos[info]:
                infos_stats[info][val]+=1
        successful_trials+=1
        size_input.append(len(x[0]))
        size_output.append(len(y[0]))

size_input=np.array(size_input)
size_output=np.array(size_output)
#n_valid_equations = sum(list(n_ops_stats.values()))
for info in infos_stats:
    non_null_points = np.where(infos_stats[info]!=0.)[0]
    values_non_null_points = infos_stats[info][non_null_points]
    count = values_non_null_points.sum()
    infos_stats[info] = np.concatenate([np.expand_dims(non_null_points,-1), np.expand_dims(values_non_null_points/count,-1)],-1)

print("Percentage of successful generation: {}".format(successful_trials/k_trials*100))
print("Time: {}".format(time.time()-deb))
print(infos_stats)
print("Average sizes: input {}, output {}".format(size_input.max(), size_output.max()))


AttributeError: type object 'params' has no attribute 'extra_unary_operators'

In [ ]:
python train.py --export_data True --dump_path=/private/home/pakamienny/Research_2/recur_datasets/all_domain/ --cpu True --max_epoch 2 --eval_input_length_modulo 5 --epoch_size 10000 

In [ ]:
x = np.array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 566.
             ])
np.argwhere(x!=0.)

# Checking encoder

In [ ]:
class params:
    int_base = 10
    has_separator = False
    
encoder = Equation(params)
print(tree.infix())
encoded = encoder.encode(tree)
print(encoded)
decoded = encoder.decode(encoded)
print(decoded.infix())

In [ ]:
class params:
    a=1
    
from src.envs.encoders import RealSeries

encoder = RealSeries(params)
x = series
print(x)
encoded = encoder.encode(x)
print(encoded)
decoded = encoder.decode(encoded)
print(decoded)